<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/VoiD_with_PySpark_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Installing SPARK for Playing


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 48.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f0aa46409776f50032940f12da16f98b021b9109f6b5cab9a9459db5d8cad9b4
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
!pip install -q findspark
import findspark
findspark.init()


# SPARK Installed... Start playing

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [23]:
!wget -q https://storage.googleapis.com/public_lddm_data/small_page_links.nt
!ls
!wget -q "https://data.dws.informatik.uni-mannheim.de/structureddata/2021-12/quads/classspecific/schema_Mountain/part_0.gz"
!ls

part_0.gz    small_page_links.nt    small_page_links.nt.2
sample_data  small_page_links.nt.1
part_0.gz    sample_data	  small_page_links.nt.1
part_0.gz.1  small_page_links.nt  small_page_links.nt.2


In [24]:
lines = spark.read.text("part_0.gz").rdd.map(lambda r: r[0])



In [25]:
lines.take(5)

['_:nb53a24408607424384c1357880ce1bc7xb0 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Mountain> <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb1 <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb2 <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb3 <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb4 <https://peakery.com/dragontail-peak-washington/>   .']

In [26]:
lines.count()

2113514

In [27]:
import re
def parseTriple(line) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', line)
    return parts[0], parts[1], parts[2],parts[3]
triples=lines.map(lambda x:parseTriple(x))
triples.count()
triples.take(1)

[('_:nb53a24408607424384c1357880ce1bc7xb0',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://schema.org/Mountain>',
  '<https://peakery.com/dragontail-peak-washington/>')]

# Exercice: Computing the VoiD for the Nt File with PySpark RDD

Write RDD PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

- QB1 CONSTRUCT { <D> v:triples ?x }
WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

- QB2 CONSTRUCT { <D> v:classes ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

- QB3 CONSTRUCT { <D> v:properties ?x }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

- QB4 CONSTRUCT { <D> v:distinctSubjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

- QB5 CONSTRUCT { <D> v:distinctObjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [28]:
print(triples.map(lambda x: x[0]).distinct().count())
print(triples.map(lambda x: x[1]).distinct().count())
print(triples.map(lambda x: x[2]).distinct().count())
print(triples.filter(lambda x: x[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>").map(lambda x: x[2]).distinct().count())


487829
253
829323
304


- QC1 CONSTRUCT { <D> v:classPartition [ v:class ?c ] } WHERE { ?s a ?c }

- QC2 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:triples ?x ] }
WHERE { SELECT (COUNT(?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }
- QC3 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:classes ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?d) AS ?x) ?c WHERE { ?s a ?c , ?d } GROUP BY ?c }
- QC4 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:properties ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c } 
- QC5 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c WHERE { ?s a ?c } GROUP BY ?c }
- QC6 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }

In [31]:
#QC2
toto=triples.filter(lambda x: x[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>")
joined=toto.join(triples)
joined.take(5)

[('_:nb53a24408607424384c1357880ce1bc7xb2',
  ('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>')),
 ('_:nb53a24408607424384c1357880ce1bc7xb2',
  ('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://schema.org/name>')),
 ('_:nb53a24408607424384c1357880ce1bc7xb2',
  ('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://schema.org/value>')),
 ('_:nb53a24408607424384c1357880ce1bc7xb3',
  ('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>')),
 ('_:nb53a24408607424384c1357880ce1bc7xb3',
  ('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://schema.org/name>'))]

- QD1 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ] } WHERE { ?s ?p ?o }
- QD2 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:triples ?x ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD3 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD4 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }

In [ ]:
Your code ??

- QE1 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:propertyPartition [ v:property ?p ] ] }
WHERE { ?s a ?c ; ?p ?o }
- QE2 CONSTRUCT { <D> v:classPartition [ v:class ?c
v:propertyPartition [ v:property ?p ; v:triples ?x ] ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE3 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctSubjects ?x ] ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE4 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctObjects ?x ; v:property ?p ] ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }

In [ ]:
Your code

# Same Exercice with PySpark SQL



In [ ]:
from pyspark.sql import Row
spo=triples.map(lambda x: Row(subject=x[0],predicate=x[1],object=x[2]))
df=spark.createDataFrame(spo)
df.createOrReplaceTempView("SPO")
df.show()



+--------------------+--------------------+--------------------+
|             subject|           predicate|              object|
+--------------------+--------------------+--------------------+
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<ht

In [ ]:

result=spark.sql("select subject from SPO")
result.show()

+--------------------+
|             subject|
+--------------------+
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
+--------------------+
only showing top 20 rows



Write SQL PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

QB1 CONSTRUCT { v:triples ?x } WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

QB2 CONSTRUCT { v:classes ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

QB3 CONSTRUCT { v:properties ?x } WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

QB4 CONSTRUCT { v:distinctSubjects ?x } WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

QB5 CONSTRUCT { v:distinctObjects ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [ ]:
Your Code